# Прв парцијален испит по Основи на Вештачката Интелигенција

## Задача 2

Разгледуваме игра која се игра самостојно. Правилата на играта се:
- Се игра на табла со димензии `N` по `N`.
- Има празни и полни кругчиња, по едно кругче за секое поле од таблата.
- Се игра исклучиво со притискање на кругчињата.
- Ако се притисне некое кругче:
    - Се менува од празно во полно, или пак обратно, од полно во празно.
    - Неговите соседи се менуваат од празно во полно, или пак обратно, од полно во празно.
    - Соседи на едно кругче се соседните кругчиња кои се наоѓаат над, под, лево и десно.
- Целта на играта е сите кругчиња да се празни.

Следи интерактивна апликација за оваа игра.

In [1]:
from plotly import graph_objects as go

In [2]:
def expand_square(square):
    neighbour_squares = []
    x, y = square
    for x, y in [(x, y), (x+1, y), (x-1, y), (x, y+1), (x, y-1)]:
        if 0 <= x < N and 0 <= y < N:
            neighbour_squares.append((x, y))
    return neighbour_squares

In [3]:
def update_fig(trace, points, selector):
    clicked_square = points.xs[0], points.ys[0]
    for x, y in expand_square(clicked_square):
        TABLE[y][x] = 0 if TABLE[y][x] else 1
    symbols = ['circle' if n else 'circle-open' for row in TABLE for n in row]
    fig.data[0].marker['symbol'] = symbols

def create_fig():
    fig = go.FigureWidget()
    x = [x for y in range(N) for x in range(N)]
    y = [y for y in range(N) for x in range(N)]
    symbols = ['circle' if n else 'circle-open' for row in TABLE for n in row]
    fig.add_scatter(x=x, y=y, mode='markers', marker_size=48, 
                    marker_symbol=symbols, marker_color='LightSkyBlue',
                    marker_line_width=6, marker_line_color='MediumPurple')
    fig.data[0].on_click(update_fig)
    fig.update_xaxes(range=[-0.5, N - 0.5], dtick=1, title='x', side='top')
    fig.update_yaxes(range=[-0.5, N - 0.5], dtick=1, title='y', autorange='reversed')
    fig.update_layout(width=120*N, height=120*N, showlegend=False)
    return fig

N = 4
TABLE = [
    [0, 0, 1, 0],
    [0, 0, 1, 1],
    [1, 1, 0, 0],
    [0, 1, 0, 0],
]

fig = create_fig()
fig

FigureWidget({
    'data': [{'marker': {'color': 'LightSkyBlue',
                         'line': {'color': 'M…

Над овој текст треба да имате интерактивна апликација за играта како на сликата подолу. Прво извршете ги ќелиите, а потоа ако ја немате интерактивната апликација, тогаш нешто не е во ред со вашата инсталација, па побарајте помош од Стефан.

![Тука треба да видите .gif анимација, ама штом го читате ова нешто не е во ред.](images/example.gif)

**Задача:** Напишете алгоритам кој ќе стигне до целта и на некој начин ќе ја прикаже постапката, чекор по чекор, како да се стигне до решението.

## Решение

In [37]:
def unzero(state, N, zero = True):
#state - matrix of current states with either -1/1 or 0/1 values
#N - number of rows and columns
#zero - if True turns zeroes in -1s, else turns -1s in zeroes 
    if zero == True:
        for i in range(N):
            for j in range(N):
                state[i][j] = -1 if state[i][j] == 0 else state[i][j]
    else:
        for i in range(N):
            for j in range(N):
                state[i][j] = 0 if state[i][j] == -1 else state[i][j]
    return state

In [66]:
def expand_state(start, N):
    start = unzero(start, N, True)
    ts = start #temp_state
    
    new_states = []
    for i in range(N):
        for j in range(N):
            ts[i][j] *= -1
            for neighbour in expand_square((i,j)):
                ts[neighbour[0]-1][neighbour[1]-1] *= -1
            new_states.append(tuple([tuple(x) for x in unzero(ts, N, False)]))
            ts = start
    return new_states

In [68]:
s = [
    [0,1],
    [0,0]
]
print(expand_state(s, 2))

[((1, 0), (1, 1)), ((0, 0), (0, 1)), ((0, 0), (0, 1)), ((0, 0), (0, 0))]


### На испитот на се бараше визуелизацијата

In [7]:
import ipywidgets as widgets

def update_fig_2(table, x, y):
    symbols = ['circle' if n else 'circle-open' for row in table for n in row]
    fig.data[0].marker['symbol'] = symbols
    fig.data[1].x, fig.data[1].y = [x], [y]

def table_generator_function():
    from statistics import median
    for table_old, table_new in zip(solution, solution[1:]):
        x_lights = [x for x in range(N) for y in range(N) if table_old[y][x] - table_new[y][x]]
        y_lights = [y for x in range(N) for y in range(N) if table_old[y][x] - table_new[y][x]]
        yield table_old, int(median(x_lights)), int(median(y_lights))
    yield solution[-1], None, None
    yield 'end'

def clicked_button(button):    
    data = next(table_generator)
    if data != 'end':
        update_fig_2(*data)
    else:
        button.disabled = True
    
def create_button():
    button = widgets.Button(description='Чекор')
    button.on_click(clicked_button)
    return button

table_generator = table_generator_function()

button = create_button()
fig = create_fig()
fig.add_scatter(mode='markers', marker_symbol='x', marker_size=24)
button.click()
widgets.VBox([button, fig])

    'data': [{'marker': {'colo…